## DSE 203 Final Project GROUP 1 - Data Store Initialization
### Sagar, Prakhar, Laben
Code to setup the following data sets into data stores along with functions to intereact with the data store


In [1]:
#Installation Steps as required
#!pip install psycopg2-binary         #This install did work. It is on the installation page

In [2]:
#Import packages
import pandas as pd        #Used to perform EDA of the structured data
import psycopg2            #Used to connect to Postgres
import csv                 #Used to read csv file into PostGres

In [3]:
#Initialize variables

#SBIR Award Dataset Settings
#Downloaded from: https://www.sbir.gov/sbirsearch/award/all
SBIRDataSet = './input_files/award_data.csv'
tableName = 'SBIR'

#Postgres Connection Settings
#Important use ' and not " when creating variable values or there will be errors when trying to connect to the database
#pghost='localhost',           #The location of the postgres server. This setting is set to assume its on the local machine
#pgdatabase='DSE203FPG1',      #The name of the database hosting the data tables. Instructions on how to create this database is provided in a following cell
#pguser='postgres',            #The username connecting used to connect to the database
#pgpassword='postgres'         #The password used to connect the user to the database

### SBIR Data

In [4]:
#Connect to your postgres DB

#-----------------------------------------------------------
#If you don't have a local database, then follow these steps
#Run this command in the terminal
#sudo -u postgres psqlrt

#Then create the database using this command:
#CREATE DATABASE DSE203FPG1;
#Other useful Terminal commands
#/c  dse203fpg1 #connects to the database
#dt  #displays list of table
#-----------------------------------------------------------
def connect_to_db():
    conn = psycopg2.connect(
        host='localhost',
        database='dse203fpg1',
        user='postgres',
        password='postgres')

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # execute a statement
    #print('PostgreSQL database version:')
    #cur.execute('SELECT version()')

    # display the PostgreSQL database server version
    #db_version = cur.fetchone()
    #print(db_version)

    return cur, conn


In [5]:
#Create the SQL Table that will hold the CSV file content
def create_SBIR_data_table(cur, conn,tableName):
    sql = '''CREATE TABLE {}("Company" VARCHAR(255), "Award Title" VARCHAR(255), "Agency" VARCHAR(255), "Branch" VARCHAR(255), "Phase" VARCHAR(255), 
                            "Program" VARCHAR(255), "Agency Tracking Number" VARCHAR(255), "Contract" VARCHAR(255), "Proposal Award Date" DATE, 
                            "Contract End Date" DATE, "Solicitation Number" TEXT, "Solicitation Year" INTEGER, "Solicitation Close Date" DATE, 
                            "Proposal Receipt Date" DATE, "Date of Notification" DATE, "Topic Code" TEXT, "Award Year" INTEGER, "Award Amount" TEXT, 
                            "Duns" VARCHAR(255), "HUBZone Owned" CHAR(1), "Socially and Economically Disadvantaged" CHAR(1), "Woman Owned" CHAR(1), 
                            "Number Employees" INTEGER, "Company Website" TEXT, "Address1"  TEXT, "Address2" TEXT, "City" VARCHAR(120), 
                            "State" CHAR(2), "Zip" TEXT, "Abstract" TEXT, "Contact Name" VARCHAR(255), "Contact Title" TEXT, "Contact Phone" TEXT, 
                            "Contact Email" VARCHAR(255), "PI Name" VARCHAR(255), "PI Title" VARCHAR(255), "PI Phone" VARCHAR(255), "PI Email" VARCHAR(255), 
                            "RI Name" VARCHAR(255), "RI POC Name" VARCHAR(255), "RI POC Phone" VARCHAR(255));'''.format(tableName)
    #sql = '''CREATE TABLE {}("Home Base" TEXT)'''.format('BOB')   #Test Case
    cur.execute(sql) 
    conn.commit()

In [6]:
# Loop through each CSV
def insert_SBIR_data_into_table(cur, conn,SBIRDataSet,tableName):
    with open(SBIRDataSet, 'r') as f:
        reader = csv.reader(f)
        next(reader) # Skip the header row.
        for row in reader:
            row = [val if val != '' else None for val in row]
            cur.execute(
            "INSERT INTO {} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s)".format(tableName),
            row
        )
    conn.commit() 

In [7]:
#Find all content in a Table
def print_SBIR_data_table(cur, conn,tableName, limitSize):
    sql3 = '''select * from {} LIMIT {};'''.format(tableName, limitSize)
    cur.execute(sql3) 
    for i in cur.fetchall(): 
        print(i) 

    conn.commit() 

In [8]:
#queries a single table and returns a dataframe
#colName = '*' will return all columns of data
#colName = 'Abstract' will return the single column Abstract
#limitSize = -1 means return all rows
#limitSize = 0 or more integer values will return the number of rows specified
def get_SBIR_data_table(cur, conn, tableName, colNames='*', limitSize=-1):

    #Convert a list of column names into the format needed for SQL
    columnStr = ''
    if colNames == '*':
        columnStr = colNames
    elif type(colNames) == list:
        for col in colNames:
            columnStr = columnStr+'"'+col+'",'
        #remove the last comma
        columnStr = columnStr[:-1]
    elif type(colNames) == str:
        columnStr = '"'+colNames+'"'
    #print (columnStr)
    
    #Create the SQL query
    if (limitSize == -1):
        sql4 = '''select {} from {};'''.format(columnStr, tableName)
    elif (limitSize > 0):
        sql4 = '''select {} from {} LIMIT {};'''.format(columnStr, tableName, limitSize)
    else:
        sql4 = ''
    #print(sql4)
    
    #Run the query
    cur.execute(sql4) 
    #print("The number of parts: ", cur.rowcount)

    #Get the Column Names from the query (Especially if the * is used)
    column_names = [desc[0] for desc in cur.description]
    #print("COL NAMES: ",column_names)

    #Initialize the dataframe with the column names (need to do this before loading the data
    sqldf = pd.DataFrame(columns=column_names)
    #print (sqldf.info())
    
    row = cur.fetchone() #fetch's one row at a time
    i=0                  #row number of the dataframe

    #Iterate throught the sql output and add one row at a time into the pandas dataframe
    while row is not None:
        print(row)
        sqldf.loc[i] = row
        i +=1
        row = cur.fetchone()

    conn.commit() 
    
    return sqldf

In [9]:
#close connection
def close_connection_to_db(conn):
    conn.close ()

In [10]:
#Run through all structured database steps once.
#This will go through each step in the process and will only work if starting fresh each time.

#If you run this command multiple times, be sure to have a terminal open
#Then Log into postgres with the following command:
#sudo -u postgres psql
#Then connect to the database with the following command:
#\c dse203hw3
#NOTE: If you use a different database name then 'dse203hw3', then replace 'dse203hw3' with the database name of your choosing
#Now that you are connected to the database in the terminal You can use the following commands
#\dt    This will list the tables in the database
#drop table nourish;    This will remove the table.
#Once the table has been removed, then you can rerun this series of conection without error.

def init_SBIR_data(SBIRDataSet,tableName):
    #Connect to the Database
    cur, conn = connect_to_db()
    
    query = '''SELECT EXISTS (SELECT relname FROM pg_class WHERE relname = 'sbir');'''
    resp = cur.execute(query)
    rows = cur.fetchone()
    if not rows[0]: 
        #Create Database empty table for structured data set
        create_SBIR_data_table(cur, conn, tableName)

        #Insert structured data into table
        insert_SBIR_data_into_table(cur, conn,SBIRDataSet,tableName)

    #close the database connection
    close_connection_to_db(conn)

In [11]:
#%%time
# Initialize the SQL database - required only one time
#init_SBIR_data(SBIRDataSet,tableName)

In [12]:
#%%time
# Read data from SQL Database (postgres)
#limitSize = 10
#SBIR_data_list = print_SBIR_data_table(cur, conn, tableName, limitSize)

In [13]:
##Example to of getting all columns and all rows
##This will most likely error out, so don't use this example!!!

#cur, conn = connect_to_db()

#get_SBIR_data_table(cur, conn, tableName)

#close_connection_to_db(conn)

In [14]:
##Example to of getting all columns and 10 rows

#cur, conn = connect_to_db()

#sbirdf = get_SBIR_data_table(cur, conn, tableName, '*', limitSize = 2)

#close_connection_to_db(conn)

#display(sbirdf)

In [15]:
##Example to of getting 1 column and 10 rows

#cur, conn = connect_to_db()

#sbirdf = get_SBIR_data_table(cur, conn, tableName, 'Abstract', limitSize = 10)

#close_connection_to_db(conn)

#display(sbirdf)

In [16]:
##Example to of getting 1 column and 10 rows

#cur, conn = connect_to_db()

#sbirdf = get_SBIR_data_table(cur, conn, tableName, ["Abstract", "State", "Award Amount", "Date of Notification"] , limitSize = 10)

#close_connection_to_db(conn)

#display(sbirdf)